In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model

from pathlib import Path

In [12]:
project_dir = Path.cwd().parent
intrs_file = project_dir / 'data' / 'intersection_accidents.pkl'
intersections = pd.read_pickle(str(intrs_file))
intersections = intersections[intersections.accidents.str.len() > 0]
intersections.head()

,lat,lon,osmid,accidents,type,angles,roadways,highway,crossing,roundabout,stop,traffic_signal
15,38.952968,-77.025404,49715974,"[{'acc_id': 1394564, 'dist': 15.56991792944886...",1,None,"[Illinois Avenue Northwest, Hamilton Street No...",stop,False,False,True,False
16,38.952901,-77.027564,49715981,"[{'acc_id': 1008276, 'dist': 0.739816983149802...",1,None,"[Georgia Avenue Northwest, Hamilton Street Nor...",NaN,False,False,False,False
19,38.952919,-77.036054,49716007,"[{'acc_id': 3401623, 'dist': 33.70175692005437...",1,None,"[Colorado Avenue Northwest, Hamilton Street No...",NaN,False,False,False,False
50,38.833847,-77.008354,49716493,"[{'acc_id': 2426593, 'dist': 10.30576255573615...",1,None,"[South Capitol Street Southeast, Martin Luther...",NaN,False,False,False,False
51,38.832895,-77.008374,49716503,"[{'acc_id': 932962, 'dist': 4.1291234227433105...",1,None,[South Capitol Street Southeast],NaN,False,False,False,False


In [13]:
intersections.iloc[15].accidents[0]

{'acc_id': 1173818,
 'dist': 14.674135279103414,
 'severity': 2.0,
 'time': '2019-05-17 07:13:12',
 'temp': 55.0,
 'crossing': False,
 'roundabout': False,
 'stop': False,
 'traffic_signal': False,
 'night': False,
 'precipitation': False,
 'weight': 0.9998861119756028}

In [14]:
def calculate_severity(accidents):
    severe = 0
    total = 0
    for accident in accidents:
        if accident['severity'] >= 3:
            severe += 1
        total += 1
    return severe / total
    

intersections['proportion_severe_accidents'] = intersections.apply(lambda x: calculate_severity(x['accidents']), axis=1)

In [15]:
intersections.head()

,lat,lon,osmid,accidents,type,angles,roadways,highway,crossing,roundabout,stop,traffic_signal,proportion_severe_accidents
15,38.952968,-77.025404,49715974,"[{'acc_id': 1394564, 'dist': 15.56991792944886...",1,None,"[Illinois Avenue Northwest, Hamilton Street No...",stop,False,False,True,False,0.000000
16,38.952901,-77.027564,49715981,"[{'acc_id': 1008276, 'dist': 0.739816983149802...",1,None,"[Georgia Avenue Northwest, Hamilton Street Nor...",NaN,False,False,False,False,0.666667
19,38.952919,-77.036054,49716007,"[{'acc_id': 3401623, 'dist': 33.70175692005437...",1,None,"[Colorado Avenue Northwest, Hamilton Street No...",NaN,False,False,False,False,0.000000
50,38.833847,-77.008354,49716493,"[{'acc_id': 2426593, 'dist': 10.30576255573615...",1,None,"[South Capitol Street Southeast, Martin Luther...",NaN,False,False,False,False,0.000000
51,38.832895,-77.008374,49716503,"[{'acc_id': 932962, 'dist': 4.1291234227433105...",1,None,[South Capitol Street Southeast],NaN,False,False,False,False,0.000000


In [18]:
X = intersections[['crossing', 'roundabout', 'stop', 'traffic_signal']]
y = intersections['proportion_severe_accidents']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [24]:
clf_sgd = linear_model.SGDRegressor(loss='squared_loss', penalty=None,  random_state=42)
clf_sgd.fit(X_train, y_train)
score = clf_sgd.score(X_train, y_train)
print(f"SGD coefficient of determination R^2: {score}")

0.02418554140579454

In [37]:
predictions = clf_sgd.predict(X_test)
error = y_test - predictions
sq_err = error ** 2
rmse = sq_err.mean() ** 0.5
print(f'RMSE: {rmse}')

RMSE: 0.41600601532198317
